In [65]:
import numpy as np

In [66]:
n_layers = [2, 3, 3, 1] # number of neurons in each layer
L = len(n_layers) - 1 # max layer index

#print the number of neurons in each layer
for i in range(L + 1):
    print(f"Layer {i}: {n_layers[i]} neurons")


Layer 0: 2 neurons
Layer 1: 3 neurons
Layer 2: 3 neurons
Layer 3: 1 neurons


In [67]:
def initialize_weights_and_biases(n_layers):
    """
    Initialize weights and biases for a multi-layer perceptron.

    Parameters:
    n_layers (list): List containing the number of neurons in each layer.

    Returns:
    W (list): List of weight matrices for each layer.
    B (list): List of bias vectors for each layer.
    """
    L = len(n_layers) - 1 # max layer index
    W = [] # weights
    B = [] # biases

    # initialize the first (index 0) layer with None (no weights or biases)
    W.append(None)
    B.append(None)

    # initialize the weights and biases for each layer
    for i in range(1, L + 1):
        W.append(0.01 * np.random.randn(n_layers[i], n_layers[i - 1])) # weights are initialized to small random values
        B.append(np.zeros((n_layers[i], 1))) # biases can be initialized to zero

    return W, B

In [68]:
#trying out the weight and bias initialization
W, B = initialize_weights_and_biases(n_layers)
# print shapes of weights
for i in range(1, len(W)):
    print(f"W[{i}] shape=({W[i].shape})")
    # print shapes of biases
for i in range(1, len(B)):
    print(f"B[{i}] shape=({B[i].shape})")

W[1] shape=((3, 2))
W[2] shape=((3, 3))
W[3] shape=((1, 3))
B[1] shape=((3, 1))
B[2] shape=((3, 1))
B[3] shape=((1, 1))


In [91]:
def prepare_data():

    # Training data
    X = np.array([
        [2.0, 2.5],
        [3.1, 1.2],
        [1.5, 2.0],
        [4.2, 3.3],
        [0.9, 1.5],
        [3.7, 2.8],
        [2.1, 1.7],
        [4.5, 3.9],
        [1.3, 1.8],
        [3.0, 2.9]
    ])

    y = np.array([
        1 if x1**2 + x2**2 > 10 else 0 for x1, x2 in X
    ])

    # reshape y to be a column vector in place
    y = y.reshape(1, m)

    return X, y

In [92]:
# trying out the data preparation function
X, y_true = prepare_data()

# print shapes of X and y
print(f"X.shape: {X.shape}")
print(f"y_true.shape: {y_true.shape}")

# print the first 5 samples
print()
print("First 5 samples:")
for i in range(5):
    print(f"X[{i}]: {X[i]}, y_true[{i}]: {y_true[:, i]}")

print()
n = X.shape[1] # number of features
m = X.shape[0] # number of samples
print(f"Number of features: {n}")
print(f"Number of samples: {m}")

X.shape: (10, 2)
y_true.shape: (1, 10)

First 5 samples:
X[0]: [2.  2.5], y_true[0]: [1]
X[1]: [3.1 1.2], y_true[1]: [1]
X[2]: [1.5 2. ], y_true[2]: [0]
X[3]: [4.2 3.3], y_true[3]: [1]
X[4]: [0.9 1.5], y_true[4]: [0]

Number of features: 2
Number of samples: 10


In [93]:
# activation function (sigmoid)
def act(Z):
  return 1 / (1 + np.exp(-1 * Z))

# derivative of the activation function (sigmoid)
def act_prime(Z):
    sig = act(Z)
    return sig * (1 - sig)

# loss function (binary cross entropy). a is the final output of the network, y is the true label
def loss(A, Y):
    return - ( (Y * np.log(A)) + (1 - Y)*np.log(1 - A) )

# derivative of the loss function (binary cross entropy) w.r.t. final activation a.
def loss_prime(A, Y):
    return - ( (Y / A) - ((1 - Y) / (1 - A)) )

In [94]:
def feedforward(X, W, B):
    A = [X.T]  # A[0] is the transpose of the input data
    Z = [None] # Z[0] is None since there is no weighted sum f_net value for the input layer

    # loop through each layer
    for i in range(1, len(W)):
        # calculate the weighted sum of inputs
        z = W[i] @ A[i - 1] + B[i]

        Z.append(z) # store the weighted sum for later use

        A.append(act(z)) # store the activation for later use

    return A, Z # to be used in backpropagation

# binary cross-entropy cost function (a is the output of the network, y is the true label)
def cost(A, Y, m):
    losses = loss(A, Y)
    summed_losses = (1 / m) * np.sum(losses, axis=1) # sum the rows (axis=1), avg. over m samples

    # sum the losses for each sample (in case of multiple output neurons)
    return np.sum(summed_losses)


In [95]:
# trying out the feedforward function
A, Z = feedforward(X, W, B)

# print shapes of f_nets
for i in range(len(A)):
    if Z[i] is not None:
        print(f"Z[{i}] shape=({Z[i].shape})")

# print shapes of activations
for i in range(len(A)):
    print(f"A[{i}] shape=({A[i].shape})")
    if Z[i] is not None:
        print(f"Z[{i}] shape=({Z[i].shape})")

# print shapes of weights
for i in range(1, len(W)):
    print(f"W[{i}] shape=({W[i].shape})")


Z[1] shape=((20, 10))
Z[2] shape=((1, 10))
A[0] shape=((2, 10))
A[1] shape=((20, 10))
Z[1] shape=((20, 10))
A[2] shape=((1, 10))
Z[2] shape=((1, 10))
W[1] shape=((20, 2))
W[2] shape=((1, 20))


In [97]:
# trying out the cost function

y_hat = A[len(n_layers) - 1] # output (last) layer activations
cost_val = cost(y_hat, y_true, m)
print(f'Cost: {cost_val}')

Cost: 0.6931627324975782


In [102]:
def backpropogation(A, Z, W, y_true, L):
    m = A[0].shape[1] # number of samples
    L = len(W) - 1 # number of layers

    # initialize gradients
    dW = [None] * (L + 1)
    dB = [None] * (L + 1)

    # calculate the gradient of the cost function w.r.t. the output layer activation
    dA = loss_prime(A[L], y_true) # shape (n_layers[L], m)
    dZ = act_prime(Z[L]) * dA # shape (n_layers[L], m)

    # loop through each layer in reverse order
    for i in range(L, 0, -1):
        dW[i] = (1 / m) * dZ @ A[i - 1].T # shape (n_layers[i], n_layers[i - 1])
        dB[i] = (1 / m) * np.sum(dZ, axis=1, keepdims=True) # shape (n_layers[i], 1)

        # prepare for the next layer
        if i > 1:
            # calculate the gradient of the cost function w.r.t. the activation of the previous layer
            dA = W[i].T @ dZ # shape (n_layers[i - 1], m)
            dZ = act_prime(Z[i - 1]) * dA # shape (n_layers[i - 1], m)

    return dW, dB

In [103]:
# trying out the backpropagation function
dW, dB = backpropogation(A, Z, W, y_true, len(n_layers) - 1)
# print shapes of gradients
for i in range(1, len(dW)):
    print(f"dW[{i}] shape=({dW[i].shape})")
    print(f"db[{i}] shape=({dB[i].shape})")

dW[1] shape=((20, 2))
db[1] shape=((20, 1))
dW[2] shape=((1, 20))
db[2] shape=((1, 1))


In [106]:
# putting it all together, and apply gradient descent
def gradient_descent(X, y_true, W, B, L, learning_rate=0.01, epochs=1000):
    m = X.shape[0] # number of samples

    for epoch in range(epochs):
        # feedforward
        A, Z = feedforward(X, W, B)

        # calculate cost
        cost_val = cost(A[L], y_true, m)

        # backpropagation
        dW, dB = backpropogation(A, Z, W, y_true, L)

        # update weights and biases
        for i in range(1, len(W)):
            W[i] -= learning_rate * dW[i]
            B[i] -= learning_rate * dB[i]

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Cost: {cost_val}")

    return W, B

In [110]:
# putting all together, trying out the gradient descent function
n_layers = [2, 20, 1]
W, B = initialize_weights_and_biases(n_layers)
X, y_true = prepare_data()
W_trained, B_trained = gradient_descent(X, y_true, W, B, len(n_layers) - 1, learning_rate=0.01, epochs=100000)

Epoch 0, Cost: 0.6941274146232828
Epoch 100, Cost: 0.6737329298294784
Epoch 200, Cost: 0.6722296564832099
Epoch 300, Cost: 0.6716639643344555
Epoch 400, Cost: 0.6711221066378306
Epoch 500, Cost: 0.6705465967974376
Epoch 600, Cost: 0.6699239801662065
Epoch 700, Cost: 0.6692423062376304
Epoch 800, Cost: 0.6684887486449378
Epoch 900, Cost: 0.667649332299807
Epoch 1000, Cost: 0.6667088017525524
Epoch 1100, Cost: 0.6656505263788914
Epoch 1200, Cost: 0.6644564285597795
Epoch 1300, Cost: 0.6631069193254557
Epoch 1400, Cost: 0.6615808192993418
Epoch 1500, Cost: 0.6598552392035253
Epoch 1600, Cost: 0.657905398866062
Epoch 1700, Cost: 0.6557043820053452
Epoch 1800, Cost: 0.6532228600068559
Epoch 1900, Cost: 0.6504288710671448
Epoch 2000, Cost: 0.6472878032156483
Epoch 2100, Cost: 0.6437627816130853
Epoch 2200, Cost: 0.6398156716256187
Epoch 2300, Cost: 0.6354088443776269
Epoch 2400, Cost: 0.6305076893308109
Epoch 2500, Cost: 0.6250836183776733
Epoch 2600, Cost: 0.6191170666488915
Epoch 2700, Cos

In [111]:
# use trained network to make predictions
def predict(X, W, B):
    A, _ = feedforward(X, W, B)
    return A[-1] # return the output layer activations

# use the trained network to make predictions
y_pred = predict(X, W_trained, B_trained)

# print the predictions
print("Predictions:")
for i in range(y_pred.shape[1]):
    print(f"X[{i}]: {X[i]}, y_pred[{i}]: {y_pred[:, i]}")
# convert predictions to binary
y_pred_binary = (y_pred > 0.5).astype(int)
# print the binary predictions
print("Binary Predictions:")
for i in range(y_pred_binary.shape[1]):
    print(f"X[{i}]: {X[i]}, y_pred_binary[{i}]: {y_pred_binary[:, i]}")

# calculate accuracy
accuracy = np.mean(y_pred_binary == y_true)
print(f"Accuracy: {accuracy * 100:.2f}%")

Predictions:
X[0]: [2.  2.5], y_pred[0]: [0.94393406]
X[1]: [3.1 1.2], y_pred[1]: [0.96384375]
X[2]: [1.5 2. ], y_pred[2]: [0.00244822]
X[3]: [4.2 3.3], y_pred[3]: [0.99999987]
X[4]: [0.9 1.5], y_pred[4]: [1.02143563e-07]
X[5]: [3.7 2.8], y_pred[5]: [0.999999]
X[6]: [2.1 1.7], y_pred[6]: [0.08902418]
X[7]: [4.5 3.9], y_pred[7]: [0.99999995]
X[8]: [1.3 1.8], y_pred[8]: [5.7087706e-05]
X[9]: [3.  2.9], y_pred[9]: [0.99999]
Binary Predictions:
X[0]: [2.  2.5], y_pred_binary[0]: [1]
X[1]: [3.1 1.2], y_pred_binary[1]: [1]
X[2]: [1.5 2. ], y_pred_binary[2]: [0]
X[3]: [4.2 3.3], y_pred_binary[3]: [1]
X[4]: [0.9 1.5], y_pred_binary[4]: [0]
X[5]: [3.7 2.8], y_pred_binary[5]: [1]
X[6]: [2.1 1.7], y_pred_binary[6]: [0]
X[7]: [4.5 3.9], y_pred_binary[7]: [1]
X[8]: [1.3 1.8], y_pred_binary[8]: [0]
X[9]: [3.  2.9], y_pred_binary[9]: [1]
Accuracy: 100.00%
